In [ ]:
!bash drive/My\ Drive/datawithcred.sh

In [ ]:
import random
import tensorflow as tf
from pathlib import Path
from src.data.make_dataset import GetDataset
from src.core.cloud import CloudTrainer

In [ ]:
tf.random.set_seed(123)

In [ ]:
dt = GetDataset(image_dims=(224, 224))

In [ ]:
train = dt.make_balaced_dataset(majority_class="chest_xray/train/PNEUMONIA",
                                minority_class="chest_xray/train/NORMAL",
                                transformations=[
                                    dt.image_flip_left_right,
                                    dt.image_central_crop,
                                    dt.image_adjust_saturation
                                ])

In [ ]:
val = dt.make_train_data(Path("chest_xray/test"))

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir=./drive/My\ Drive/models --port=6006

In [ ]:
def conv_block(filters, kernel_size):
    c_block = tf.keras.Sequential([
        tf.keras.layers.SeparableConv2D(filters=filters,
                                        kernel_size=kernel_size,
                                        activation="relu",
                                        padding="same"),
        tf.keras.layers.SeparableConv2D(filters=filters,
                                        kernel_size=kernel_size,
                                        activation="relu",
                                        padding="same"),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPool2D()
    ])
    return c_block

In [ ]:
def model_top():
    m = tf.keras.Sequential([
        tf.keras.layers.Dense(256, activation="relu"),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dense(1, activation="sigmoid")
    ])

In [ ]:
def model_top_with_dropout(rate):
    m = tf.keras.Sequential([
        tf.keras.layers.Dense(256, activation="relu"),
        tf.keras.layers.Dropout(rate),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dropout(rate)
    ])
    return m

# Tuning number of layers

In [ ]:
def get_model():
        model = tf.keras.Sequential()
        model.add(tf.keras.Input(shape=(dt.width, dt.height, 3)))
        for i in range(1, l+1):
            FILTER_SIZE = 64 * i
            model.add(tf.keras.layers.Conv2D(filters=FILTER_SIZE,
                                        kernel_size=(3,3),
                                        activation="relu",
                                        padding="same"))
            model.add(tf.keras.layers.Conv2D(filters=FILTER_SIZE,
                                        kernel_size=(3,3),
                                        activation="relu",
                                        padding="same"))
            model.add(tf.keras.layers.BatchNormalization())
            model.add(tf.keras.layers.MaxPool2D())
        model.add(tf.keras.layers.Flatten())
        model.add(tf.keras.layers.Dense(256, activation="relu"))
        model.add(tf.keras.layers.Dropout(RATE))
        model.add(tf.keras.layers.Dense(128, activation="relu"))
        model.add(tf.keras.layers.Dropout(RATE))
        model.add(tf.keras.layers.Dense(1, activation="sigmoid"))
        return model

In [ ]:
model.summary()

In [ ]:
for l in random.sample(list(range(1, 7)), 3):
    print(f"Number of conv layer of model: {l}")
    trainer = CloudTrainer(experiment_name=f"custom-layer-{l}", save_freq=100)
    RATE = 0.3
    trainer.add_keras_seq_model(get_model)
    trainer.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
        loss="binary_crossentropy",
        metrics=["accuracy",
                    tf.keras.metrics.Precision(name='precision'),
                    tf.keras.metrics.Recall(name='recall')]
        )
    trainer.fit(
        train,
        validation_data=val,
        epochs=30
    )
    





In [ ]:
def get_model():
        model = tf.keras.Sequential()
        # model.add(tf.keras.Input(shape=(224, 224, 3)))
        for i in range(1, l+1):
            FILTER_SIZE = 64 * i
            model.add(tf.keras.layers.SeparableConv2D(filters=FILTER_SIZE,
                                        kernel_size=(3,3),
                                        activation="relu",
                                        padding="same"))
            model.add(tf.keras.layers.SeparableConv2D(filters=FILTER_SIZE,
                                        kernel_size=(3,3),
                                        activation="relu",
                                        padding="same"))
            model.add(tf.keras.layers.BatchNormalization())
            model.add(tf.keras.layers.MaxPool2D())
        model.add(tf.keras.layers.Flatten())
        model.add(tf.keras.layers.Dense(256, activation="relu"))
        model.add(tf.keras.layers.Dropout(RATE))
        model.add(tf.keras.layers.Dense(128, activation="relu"))
        model.add(tf.keras.layers.Dropout(RATE))
        model.add(tf.keras.layers.Dense(1, activation="sigmoid"))
        return model

In [ ]:
for l in random.sample(list(range(1, 7)), 3):
l = 5
print(f"Number of conv layer of model: {l}")
trainer = CloudTrainer(experiment_name=f"custom-sep-layer-{l}", save_freq=100)
RATE = 0.3
trainer.add_keras_seq_model(get_model)
trainer.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
    loss="binary_crossentropy",
    metrics=["accuracy",
                tf.keras.metrics.Precision(name='precision'),
                tf.keras.metrics.Recall(name='recall')]
    )
trainer.fit(
    train,
    validation_data=val,
    epochs=30
)
    

In [ ]:
for i in [0.1, 0.2, 0.3]:
    l=5
    name = str(i).split(".")[1] + "0"
    trainer = CloudTrainer(experiment_name=f"custom-dropout-{name}", save_freq=100)
    if i:
        print(f"Dropout rate: {name}")
        RATE = i
        trainer.add_keras_seq_model(get_model)
        trainer.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
            loss="binary_crossentropy",
            metrics=["accuracy",
                        tf.keras.metrics.Precision(name='precision'),
                        tf.keras.metrics.Recall(name='recall')]
            )
        trainer.fit(
            train,
            validation_data=val,
            epochs=30
        )
    else:
        print(f"Dropout rate: {i}")
        def get_model():
            model = tf.keras.Sequential()
            model.add(tf.keras.Input(shape=(dt.width, dt.height, 3)))
            for i in range(1, l+1):
                FILTER_SIZE = 64 * i
                model.add(tf.keras.layers.Conv2D(filters=FILTER_SIZE,
                                            kernel_size=(3,3),
                                            activation="relu",
                                            padding="same"))
                model.add(tf.keras.layers.Conv2D(filters=FILTER_SIZE,
                                            kernel_size=(3,3),
                                            activation="relu",
                                            padding="same"))
                model.add(tf.keras.layers.BatchNormalization())
                model.add(tf.keras.layers.MaxPool2D())
            model.add(tf.keras.layers.Flatten())
            model.add(tf.keras.layers.Dense(256, activation="relu"))
            model.add(tf.keras.layers.Dense(128, activation="relu"))
            model.add(tf.keras.layers.Dense(1, activation="sigmoid"))
            return model
        trainer = CloudTrainer(experiment_name=f"custom-dropout-{i}", save_freq=100)
        trainer.add_keras_seq_model(get_model)
        trainer.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
            loss="binary_crossentropy",
            metrics=["accuracy",
                        tf.keras.metrics.Precision(name='precision'),
                        tf.keras.metrics.Recall(name='recall')]
            )
        trainer.fit(
            train,
            validation_data=val,
            epochs=30
        )